### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [2]:
train, dev, test = [], [], []

In [3]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [4]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [5]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [6]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [7]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []

# Python Modules

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from keras.preprocessing import sequence
from keras.layers import Bidirectional, LSTM, Dense, Embedding, GRU, Dropout, Attention
from keras import Sequential
import pandas as pd
import numpy as np
from collections import Counter

# Model

In [9]:
# Transformer Model
class Transformer(layers.Layer):
    def __init__(self, n_heads, hidden_dim, embed_size, vocab_len, seq_len):
        super(Transformer, self).__init__()
        # Token embedding
        self.embedding = layers.Embedding(input_dim=vocab_len, output_dim=embed_size)
        # Positional Embedding
        self.positional_embedding = layers.Embedding(input_dim=seq_len, output_dim=embed_size)

        # Multihead Self Attention
        self.selfattn = layers.MultiHeadAttention(num_heads=n_heads, key_dim=embed_size)

        # Layer Normalization
        self.ln1 = layers.LayerNormalization(epsilon=1e-6)

        # Dropout
        self.d1 = layers.Dropout(0.2)
        self.ln2 = layers.LayerNormalization(epsilon=1e-6)
        self.d2 = layers.Dropout(0.2)

        # Feed Forward Neural Network
        self.ffn = keras.Sequential([layers.Dense(hidden_dim, activation="relu"), layers.Dense(embed_size),])
        
    def get_embeddings(self, input):
        seq_len = tf.shape(input)[-1]
        pos = tf.range(start=0, limit=seq_len, delta=1)
        pos = self.positional_embedding(pos)
        input = self.embedding(input)
        return input + pos
        
    
    def call(self, input):

        input_embed = self.get_embeddings(input)

        # Perform Multihead Self Attention between inputs
        attn = self.selfattn(input_embed, input_embed)
        attn = self.d1(attn)

        # Perform skip connection
        out = input_embed + attn

        # Perform normalization
        out = self.ln1(out)

        # Feed Forward Network
        output = self.ffn(out)
        output = self.d2(output)

        # Perform skip connection and normalization
        output = self.ln2(out + output)
        return output


# Constants

In [10]:
# CONSTANTS
MAX_SEQUENCE_LENGTH = 100
VOCAB_SIZE = 15000
EMBEDDING_SIZE = 100
FORWARD_DIMS = 100
N_HEADS = 2
################

# Find top 15k words

In [11]:
df_train = pd.DataFrame(train, columns=['class', 'input'])
df_val = pd.DataFrame(dev, columns=['class', 'input'])

# Identify the most common words
x = Counter(" ".join(df_train["input"]).split()).most_common(VOCAB_SIZE)

# Put the words in the list
words =[]
for i in range(len(x)):
  words.append(x[i][0])

# Tokenize the Input

In [12]:
# Tokenize the words
def tokenizer(x):
  x = x.rstrip('\n\r').split()
  x = [i for i in x if i not in [',', '.', ':', ';', '?','!']]
  return x

# Replace each token with the token-id
def tokenize(x):
  tokens = [words.index(w) if w in words else 0 for w in x]
  return tokens


df_train['input']= df_train['input'].apply(tokenizer)
df_train['input'] = df_train['input'].apply(tokenize)

# Pad the sequences
x_train = sequence.pad_sequences(df_train['input'], MAX_SEQUENCE_LENGTH)
y_train = df_train['class']

# Do the same for the development data
df_val['input'] = df_val['input'].apply(tokenizer)
df_val['input'] = df_val['input'].apply(tokenize)

x_val = sequence.pad_sequences(df_val['input'], MAX_SEQUENCE_LENGTH)
y_val = df_val['class']

In [13]:
ip = layers.Input(shape=(MAX_SEQUENCE_LENGTH,))


trans = Transformer(N_HEADS, FORWARD_DIMS, EMBEDDING_SIZE, VOCAB_SIZE, MAX_SEQUENCE_LENGTH)

# Transformer
input = trans(ip)
input = layers.GlobalAveragePooling1D()(input)
input = layers.Dropout(0.3)(input)

# Fully Connected Layer-1
input = layers.Dense(150, activation="relu")(input)
input = layers.Dropout(0.3)(input)

# Fully Connected Layer-2
input = layers.Dense(150, activation="relu")(input)
input = layers.Dropout(0.3)(input)

# Final Prediction
op = layers.Dense(2, activation="softmax")(input)

model = keras.Model(inputs=ip, outputs=op)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Training

In [14]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data = (x_val, y_val))

Epoch 1/10
507/507 [==============================] - 74s 142ms/step - loss: 0.7075 - accuracy: 0.5055 - val_loss: 0.6936 - val_accuracy: 0.4697
Epoch 2/10
507/507 [==============================] - 72s 141ms/step - loss: 0.6687 - accuracy: 0.5501 - val_loss: 0.5069 - val_accuracy: 0.7588
Epoch 3/10
507/507 [==============================] - 71s 140ms/step - loss: 0.3927 - accuracy: 0.8372 - val_loss: 0.3662 - val_accuracy: 0.8535
Epoch 4/10
507/507 [==============================] - 71s 139ms/step - loss: 0.2011 - accuracy: 0.9250 - val_loss: 0.3656 - val_accuracy: 0.8629
Epoch 5/10
507/507 [==============================] - 71s 140ms/step - loss: 0.1274 - accuracy: 0.9509 - val_loss: 0.4788 - val_accuracy: 0.8619
Epoch 6/10
507/507 [==============================] - 70s 139ms/step - loss: 0.0890 - accuracy: 0.9611 - val_loss: 0.5311 - val_accuracy: 0.8703
Epoch 7/10
507/507 [==============================] - 70s 138ms/step - loss: 0.0696 - accuracy: 0.9683 - val_loss: 0.6450 - val_ac

# Prediction

In [15]:
# Tokenize test data
df_test = pd.DataFrame(test, columns=['input'])
df_test['input'] = df_test['input'].apply(tokenizer)
df_test['input'] = df_test['input'].apply(tokenize)

x_test = sequence.pad_sequences(df_test['input'], MAX_SEQUENCE_LENGTH)

In [16]:
# Append the predictions in the list
results=[]
r = model.predict(x_test, batch_size = 32)
res = np.argmax(r, axis=1)
for i in range(len(res)):
  results.append(res[i])

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [17]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [18]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [19]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')